In [ ]:
pip install mtcnn

In [ ]:
import cv2
import os
import numpy as np
from mtcnn import MTCNN

def crop_faces_from_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    detector = MTCNN()
    for class_name in os.listdir(input_folder):
        class_path = os.path.join(input_folder, class_name)

        if not os.path.isdir(class_path):
            continue

        output_class_folder = os.path.join(output_folder, class_name)
        os.makedirs(output_class_folder, exist_ok=True)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Failed to load image: {img_path}")
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            detections = detector.detect_faces(img_rgb)

            if not detections:
                print(f"No faces detected in image: {img_path}")
                continue

            for i, detection in enumerate(detections):
                x, y, width, height = detection['box']
                x, y, width, height = int(x), int(y), int(width), int(height)
                cropped_face = img[y:y+height, x:x+width]

                output_file = os.path.join(output_class_folder, f'{img_name}_face_{i}.jpg')
                cv2.imwrite(output_file, cropped_face)
                print(f'Face {i} cropped and saved as "{output_file}"')

input_folder = '/content/drive/MyDrive/hollywood_celeb_male'
output_folder = ''
crop_faces_from_folder(input_folder, output_folder)


Face 0 cropped and saved as "/content/drive/MyDrive/hollywood_celeb_male_cropped/Will Smith/062_07fcfec7.jpg_face_0.jpg"
Face 0 cropped and saved as "/content/drive/MyDrive/hollywood_celeb_male_cropped/Will Smith/023_cb306a1a.jpg_face_0.jpg"
Face 0 cropped and saved as "/content/drive/MyDrive/hollywood_celeb_male_cropped/Will Smith/014_f512d81c.jpg_face_0.jpg"
Face 0 cropped and saved as "/content/drive/MyDrive/hollywood_celeb_male_cropped/Will Smith/073_f04cd664.jpg_face_0.jpg"
Face 0 cropped and saved as "/content/drive/MyDrive/hollywood_celeb_male_cropped/Will Smith/053_8405b30f.jpg_face_0.jpg"
Face 0 cropped and saved as "/content/drive/MyDrive/hollywood_celeb_male_cropped/Will Smith/056_abd6e06b.jpg_face_0.jpg"
Face 0 cropped and saved as "/content/drive/MyDrive/hollywood_celeb_male_cropped/Will Smith/054_cbee29ae.jpg_face_0.jpg"
Face 0 cropped and saved as "/content/drive/MyDrive/hollywood_celeb_male_cropped/Will Smith/046_6d5b1ed6.jpg_face_0.jpg"
Face 0 cropped and saved as "/co

In [ ]:
import tensorflow as tf
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

img_height = 224
img_width = 224

base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, pooling='avg')

def load_and_preprocess_image(img_path):

    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array = preprocess_input(img_array)  
    return img_array

def get_embeddings(model, img_array):
    embeddings = model.predict(img_array)
    return embeddings

def prepare_dataset_embeddings(train_dir):
    image_paths = []
    labels = []
    embeddings = []

    for class_dir in os.listdir(train_dir):
        class_path = os.path.join(train_dir, class_dir)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img_array = load_and_preprocess_image(img_path)
                embedding = get_embeddings(base_model, img_array)
                embeddings.append(embedding[0])
                labels.append(class_dir)
                image_paths.append(img_path)

    return np.array(embeddings), np.array(labels), image_paths


In [ ]:

train_dir = '/content/drive/MyDrive/hollywood_celeb_male_cropped'
embeddings, labels, image_paths = prepare_dataset_embeddings(train_dir)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

knn_classifier = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn_classifier.fit(embeddings, encoded_labels)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━

KNeighborsClassifier(metric='euclidean')

In [ ]:
import joblib
model_path = '/content/drive/MyDrive/celebrity_look_alike_male_knn_model.joblib'
joblib.dump(knn_classifier, model_path)
path_to_save = '/content/drive/MyDrive/celebrity_look_alike_male_label_encoder.pkl'
joblib.dump(label_encoder, path_to_save)

['/content/drive/MyDrive/celebrity_look_alike_male_knn_model.joblib']